# 📊 Model Evaluation & Validation

**Goal**: Assess clustering quality and interpret risk profiles.

**Evaluation metrics**:
1. **Silhouette Score**: How well-separated are clusters?
2. **Cluster profiles**: Do they make business sense?
3. **Sector distribution**: Are certain sectors naturally riskier?
4. **Feature importance**: Which features drive the clustering?

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import silhouette_score, silhouette_samples
import sys
sys.path.append('../src')
from clustering import StockClusterer

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

## 1️⃣ Load Data and Model

In [ ]:
df = pd.read_csv('../Data/Processed/nse_clustered.csv')
clusterer = StockClusterer.load_model('../models/stock_clusterer.pkl')

print(f"Loaded {len(df)} clustered stocks")
print(f"Model uses {len(clusterer.feature_columns)} features")

## 2️⃣ Silhouette Analysis

**Silhouette score**: Ranges from -1 to 1
- **> 0.7**: Excellent separation
- **0.5-0.7**: Good separation
- **0.3-0.5**: Moderate separation
- **< 0.3**: Weak separation

In [ ]:
# Calculate overall silhouette
X = df[clusterer.feature_columns].fillna(df[clusterer.feature_columns].median())
X_scaled = clusterer.scaler.transform(X)

avg_silhouette = silhouette_score(X_scaled, df['Cluster'])
print(f"\n📊 Overall Silhouette Score: {avg_silhouette:.3f}")

if avg_silhouette >= 0.5:
    print("✅ EXCELLENT - Clusters are well-separated!")
elif avg_silhouette >= 0.4:
    print("✓ GOOD - Clusters have clear structure")
elif avg_silhouette >= 0.3:
    print("⚠ MODERATE - Some overlap between clusters")
else:
    print("❌ WEAK - Clusters poorly separated")

In [ ]:
# Per-cluster silhouette scores
sample_silhouette = silhouette_samples(X_scaled, df['Cluster'])

fig, ax = plt.subplots(figsize=(10, 7))
y_lower = 10
colors = ['green', 'blue', 'orange', 'red']

for i in range(4):
    cluster_silhouettes = sample_silhouette[df['Cluster'] == i]
    cluster_silhouettes.sort()
    
    size = cluster_silhouettes.shape[0]
    y_upper = y_lower + size
    
    ax.fill_betweenx(np.arange(y_lower, y_upper),
                     0, cluster_silhouettes,
                     facecolor=colors[i], edgecolor=colors[i], alpha=0.7)
    
    ax.text(-0.05, y_lower + 0.5 * size, f"C{i}")
    y_lower = y_upper + 10

ax.set_title("Silhouette Plot for Each Cluster", fontsize=14, fontweight='bold')
ax.set_xlabel("Silhouette Coefficient", fontsize=12)
ax.set_ylabel("Cluster Label", fontsize=12)
ax.axvline(x=avg_silhouette, color="red", linestyle="--", linewidth=2, label=f"Average: {avg_silhouette:.3f}")
ax.axvline(x=0, color="black", linestyle="-", linewidth=0.5)
ax.legend()
plt.tight_layout()
plt.show()

## 3️⃣ Cluster Characteristics

Compare clusters across key risk dimensions

In [ ]:
summary = clusterer.get_cluster_summary(df)
print("\n📈 Cluster Profiles:")
print(summary)

# Visualize profiles
fig, axes = plt.subplots(2, 3, figsize=(16, 10))
axes = axes.flatten()

metrics = ['volatility_mean', 'sharpe_ratio', 'max_drawdown', 
          'trading_frequency', 'rsi_mean', 'downside_deviation']
titles = ['Volatility', 'Sharpe Ratio', 'Max Drawdown', 
         'Trading Frequency', 'RSI', 'Downside Deviation']

for idx, (metric, title) in enumerate(zip(metrics, titles)):
    if metric in df.columns:
        df_plot = df.groupby('Risk_Profile')[metric].mean().reindex(
            ['Low Risk', 'Medium-Low Risk', 'Medium-High Risk', 'High Risk']
        )
        
        axes[idx].bar(range(len(df_plot)), df_plot.values, 
                     color=['green', 'blue', 'orange', 'red'], alpha=0.7, edgecolor='black')
        axes[idx].set_title(title, fontweight='bold', fontsize=12)
        axes[idx].set_xticks(range(4))
        axes[idx].set_xticklabels(['Low', 'Med-Low', 'Med-High', 'High'], rotation=45)
        axes[idx].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

## 4️⃣ Sector Distribution

Are certain sectors naturally riskier?

In [ ]:
if 'Sector' in df.columns:
    sector_risk = pd.crosstab(df['Sector'], df['Risk_Profile'], normalize='index') * 100
    sector_risk = sector_risk[['Low Risk', 'Medium-Low Risk', 'Medium-High Risk', 'High Risk']]
    
    print("\n🏭 Sector Risk Distribution (%):")
    print(sector_risk.round(1))
    
    # Plot heatmap
    plt.figure(figsize=(12, 8))
    sns.heatmap(sector_risk, annot=True, fmt='.1f', cmap='RdYlGn_r', 
                cbar_kws={'label': '% of Sector'}, linewidths=0.5)
    plt.title('Risk Profile Distribution by Sector', fontsize=14, fontweight='bold')
    plt.ylabel('Sector', fontsize=12)
    plt.xlabel('Risk Profile', fontsize=12)
    plt.tight_layout()
    plt.show()

## 5️⃣ Feature Importance

Which features distinguish high-risk from low-risk stocks?

In [ ]:
# Compare high vs low risk
high_risk = df[df['Risk_Profile'] == 'High Risk'][clusterer.feature_columns].mean()
low_risk = df[df['Risk_Profile'] == 'Low Risk'][clusterer.feature_columns].mean()

diff = ((high_risk - low_risk) / (low_risk.abs() + 1e-10) * 100).sort_values(ascending=False)

plt.figure(figsize=(12, 8))
colors = ['red' if x > 0 else 'green' for x in diff.values]
plt.barh(range(len(diff)), diff.values, color=colors, alpha=0.7, edgecolor='black')
plt.yticks(range(len(diff)), diff.index, fontsize=10)
plt.xlabel('% Difference (High Risk vs Low Risk)', fontsize=12, fontweight='bold')
plt.title('Feature Differences: High Risk vs Low Risk', fontsize=14, fontweight='bold')
plt.axvline(x=0, color='black', linestyle='-', linewidth=1)
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

print("\n🔍 Top 5 Features Separating Risk Levels:")
print(diff.head(5))

## 6️⃣ Sample Stocks by Risk Profile

In [ ]:
for risk_profile in ['Low Risk', 'Medium-Low Risk', 'Medium-High Risk', 'High Risk']:
    subset = df[df['Risk_Profile'] == risk_profile]
    
    if len(subset) > 0:
        print(f"\n{'='*70}")
        print(f"{risk_profile.upper()} - Top 5 Stocks")
        print('='*70)
        
        cols = ['Stock_code', 'Name', 'Sector', 'volatility_mean', 'sharpe_ratio', 'max_drawdown']
        available_cols = [c for c in cols if c in subset.columns]
        
        # Get 5 most stable in each category
        sample = subset.nsmallest(5, 'volatility_mean')[available_cols]
        print(sample.to_string(index=False))

---

## 📚 Key Takeaways

### ✅ Model Performance
- **Silhouette Score**: Indicates cluster quality
- **Cluster Balance**: Check if clusters have reasonable sizes
- **Feature Importance**: See which metrics matter most

### 📊 Business Insights
- **Low Risk**: Stable, liquid, consistent returns
- **Medium Risk**: Balanced risk/reward profiles
- **High Risk**: Volatile, potential for high returns but big drawdowns

### 🎯 Investor Application
- **Conservative investors**: Focus on Low/Medium-Low clusters
- **Aggressive investors**: Consider Medium-High/High clusters
- **Diversification**: Mix stocks from different clusters

**Next**: Generate insights and identify limitations! 🎯